# Limpeza, Tratamento e Pré-Processamento dos Dados para o Modelo

## 1. Imports

In [2]:
#from IPython.display import display, Markdown, Image
import numpy as np
import pandas as pd

from fuzzywuzzy import process


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

#metricas
from sklearn.metrics import classification_report, confusion_matrix

#modelos sekeção de atributos
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

#rede neural
import tensorflow as tf


import joblib
import yaml
import psycopg2

import random as python_random

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

# from utils import *

#import inflection

colors = ["#9467bd", "#057476", "#FF7A00"]
sns.set_style("whitegrid")
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette(colors))

In [3]:
# reprodutividade
seed = 41
np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)

## 1.1 Coleta dos Dados

In [4]:
df_raw = pd.read_csv('../2-coleta/database.csv')

In [5]:
df_raw.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660,Outros,Ens.Médio,MuitoBom,57,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21,46557,Outros,Ens.Médio,MuitoBom,37,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13,43939,Própria,Ens.Médio,Bom,22,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10,37262,Própria,Superior,Baixo,35,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6,52606,Própria,PósouMais,Justo,26,0,Casado,TrailConqueror,199564.0,400000.0,ruim


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   profissao        147 non-null    object 
 1   tempoprofissao   150 non-null    int64  
 2   renda            150 non-null    int64  
 3   tiporesidencia   147 non-null    object 
 4   escolaridade     150 non-null    object 
 5   score            150 non-null    object 
 6   idade            150 non-null    int64  
 7   dependentes      150 non-null    int64  
 8   estadocivil      150 non-null    object 
 9   produto          150 non-null    object 
 10  valorsolicitado  150 non-null    float64
 11  valortotalbem    150 non-null    float64
 12  classe           150 non-null    object 
dtypes: float64(2), int64(4), object(7)
memory usage: 15.4+ KB


# 2. Tratamento de Nulos

- Algoritmos de Machine Learning, especialmente RNA, não conseguem lidar com valores nulos.

O que devemos fazer:
1. Remover a linha;
Essa possibilidade é útil quando possui poucas ocorrências em poucas colunas, porém geralmente esse é último caso de decisão
2. Substituir a ocorrência, mantendo a linha;
É um processo mais comum. utilizando a média ou a mediana.

Regra para o processo 2:
Se os dados forem categóricos:
    - Moda
Se os Dados forem numéricos:
    - Mediana


In [7]:
df_raw1 = df_raw.copy()

In [8]:
df_raw1.head()


,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660,Outros,Ens.Médio,MuitoBom,57,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21,46557,Outros,Ens.Médio,MuitoBom,37,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13,43939,Própria,Ens.Médio,Bom,22,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10,37262,Própria,Superior,Baixo,35,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6,52606,Própria,PósouMais,Justo,26,0,Casado,TrailConqueror,199564.0,400000.0,ruim


In [9]:

def substitui_nulos(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            moda = df[coluna].mode()[0]
            df[coluna].fillna(moda, inplace=True)
        else:
            mediana = df[coluna].median()
            df[coluna].fillna(mediana, inplace=True)

     

In [10]:
substitui_nulos(df_raw1)

C:\Users\Yago\AppData\Local\Temp\ipykernel_10092\1477909649.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[coluna].fillna(moda, inplace=True)
C:\Users\Yago\AppData\Local\Temp\ipykernel_10092\1477909649.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when do

In [34]:
df_raw1.isnull().values.any()

False

## 3. Tratamento de Erros de Entrada (Digitação)

- Vamos usar o domínio da categoria: valores possíveis
- Não é possível fazer função genérica e universal
Obs:  Valores fora do domínio podem gerar erro no modelo

In [12]:
df_raw2 = df_raw1.copy()

In [13]:
# Exibir todas as linhas do DataFrame
pd.set_option('display.max_rows', None)

#df_raw2.head(250) #ou
#print(df_raw2)
print(df_raw2.columns)

# Voltar ao limite padrão de linhas
pd.reset_option('display.max_rows')

Index(['profissao', 'tempoprofissao', 'renda', 'tiporesidencia',
       'escolaridade', 'score', 'idade', 'dependentes', 'estadocivil',
       'produto', 'valorsolicitado', 'valortotalbem', 'classe'],
      dtype='object')


In [14]:
profissoes_validas = ['Advogado', 'Arquiteto', 'Cientista de Dados', 'Contador',
                      'Dentista', 'Empresário', 'Engenheiro', 'Médico', 'Programador']

def corrigir_erros_digitacao(df, coluna, lista_valida):
    for i, valor in enumerate(df[coluna]):
        valor_str = str(valor) if pd.notnull(valor) else valor
        
        if valor_str not in lista_valida and pd.notnull(valor_str):
            correcao = process.extractOne(valor_str, lista_valida)[0]
            df.at[i, coluna] = correcao

In [15]:
corrigir_erros_digitacao(df_raw2, 
                         'profissao',
                         profissoes_validas)

In [39]:
print(f"Antes:  {df_raw1['profissao'].unique()}")
print(f"Depois: {df_raw2['profissao'].unique()}")



Antes:  ['Cientista de Dados' 'Empresário' 'Dentista' 'Engenheiro' 'Contador'
 'Arquiteto' 'Programador' 'Advogado' 'Adv.' 'Médico' 'Dent.']
Depois: ['Cientista de Dados' 'Empresário' 'Dentista' 'Engenheiro' 'Contador'
 'Arquiteto' 'Programador' 'Advogado' 'Médico']


## 4. Tratamento de Outliers

Abordagem estística: Desvio padrão ou intervalo interquartil
Abordagem de negócio: Domínio do atríbuto

    - Tempo de Profissão: 0-70
    - Idade: 0-110


In [18]:
df_raw3 = df_raw2.copy()

In [19]:
def tratar_outliers(df, coluna, minimo, maximo):
    mediana = df[(df[coluna] >= minimo) & (df[coluna] <= maximo)][coluna].median()
    
    df[coluna] = df[coluna].apply(lambda x: mediana if x < minimo or x > maximo else x)
    return df 

In [20]:
tratar_outliers(df_raw3,
                'tempoprofissao',
                0,
                70)

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24.0,58660,Outros,Ens.Médio,MuitoBom,57,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21.0,46557,Outros,Ens.Médio,MuitoBom,37,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13.0,43939,Própria,Ens.Médio,Bom,22,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10.0,37262,Própria,Superior,Baixo,35,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6.0,52606,Própria,PósouMais,Justo,26,0,Casado,TrailConqueror,199564.0,400000.0,ruim
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Médico,36.0,47480,Própria,Superior,Bom,63,0,Divorciado,SpeedFury,217011.0,800000.0,bom
146,Advogado,39.0,20860,Alugada,Ens.Fundamental,Bom,36,0,Víuvo,DoubleDuty,139244.0,320000.0,ruim
147,Arquiteto,26.0,31394,Própria,PósouMais,MuitoBom,53,1,Divorciado,ElegantCruise,107035.0,300000.0,ruim
148,Médico,19.0,39769,Própria,Ens.Médio,Baixo,63,0,Solteiro,VoyageRoamer,54520.0,350000.0,bom


In [21]:
tratar_outliers(df_raw3,
                'idade',
                0,
                110)

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24.0,58660,Outros,Ens.Médio,MuitoBom,57.0,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21.0,46557,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13.0,43939,Própria,Ens.Médio,Bom,22.0,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10.0,37262,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6.0,52606,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.0,400000.0,ruim
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Médico,36.0,47480,Própria,Superior,Bom,63.0,0,Divorciado,SpeedFury,217011.0,800000.0,bom
146,Advogado,39.0,20860,Alugada,Ens.Fundamental,Bom,36.0,0,Víuvo,DoubleDuty,139244.0,320000.0,ruim
147,Arquiteto,26.0,31394,Própria,PósouMais,MuitoBom,53.0,1,Divorciado,ElegantCruise,107035.0,300000.0,ruim
148,Médico,19.0,39769,Própria,Ens.Médio,Baixo,63.0,0,Solteiro,VoyageRoamer,54520.0,350000.0,bom


In [41]:
print(df_raw2.describe())

       tempoprofissao         renda       idade  dependentes  valorsolicitado  \
count      150.000000    150.000000  150.000000   150.000000       150.000000   
mean      6690.573333  36406.813333   47.393333     0.980000    144512.680000   
std      81647.612861  12974.282533   20.038184     0.993063    113913.175165   
min          0.000000   7814.000000   21.000000     0.000000     28290.000000   
25%         13.000000  24271.750000   35.250000     0.000000     69172.000000   
50%         24.000000  35795.000000   46.000000     1.000000    123258.000000   
75%         32.750000  46361.000000   57.000000     2.000000    170513.750000   
max     999999.000000  59976.000000  224.000000     4.000000    800000.000000   

       valortotalbem  
count     150.000000  
mean   375161.993333  
std    178933.034924  
min     31170.000000  
25%    280000.000000  
50%    320000.000000  
75%    400000.000000  
max    800000.000000  


In [45]:
print(df_raw3.describe())

       tempoprofissao         renda       idade  dependentes  valorsolicitado  \
count      150.000000    150.000000  150.000000   150.000000       150.000000   
mean        22.900000  36406.813333   46.206667     0.980000    144512.680000   
std         11.114867  12974.282533   13.812890     0.993063    113913.175165   
min          0.000000   7814.000000   21.000000     0.000000     28290.000000   
25%         13.000000  24271.750000   35.250000     0.000000     69172.000000   
50%         24.000000  35795.000000   46.000000     1.000000    123258.000000   
75%         32.000000  46361.000000   57.000000     2.000000    170513.750000   
max         40.000000  59976.000000   70.000000     4.000000    800000.000000   

       valortotalbem  
count     150.000000  
mean   375161.993333  
std    178933.034924  
min     31170.000000  
25%    280000.000000  
50%    320000.000000  
75%    400000.000000  
max    800000.000000  


## 5. Engenharia de Atributos

- Codificação de Variáveis Categóricas
- Normalização e Padronização
- Tratamentos de Valores Ausentes
- Interactions


In [46]:
df_raw4 = df_raw3.copy()

### 5.1 Criação de novos Atributos (Interactions)

In [47]:
df_raw4['proporcaosolicitadototal'] = df_raw4['valorsolicitado']/df_raw4['valortotalbem']
df_raw4['proporcaosolicitadototal'] = df_raw4['proporcaosolicitadototal'].astype(float)


## Divisão em treino e teste

É uma boa prática realizar a divisão dos dados em treino e teste antes da normalização e categorização, por conta do vazamento de dados

In [48]:
# Dividindo os Dados
X = df_raw4.drop('classe', axis=1)
y = df_raw4['classe']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = seed)



### 5.2 Normalização dos Dados
- Processo de transformação de dados numéticos
- Variáveis em escalas diferentes:
Contribuem de forma desbalanceada para o modelo
Exemplo: Sálario e Altura

Existem técnicas de machine learning que não é necessário realizar a normalização dos Dados:
- Árvore de Decisão

Normalização

In [49]:
# Normalização de Dados
# Pode-se também realizar um teste lógico com if e capturar as colunas númericas
espaçamento = '-'
def save_scalers(df, nome_colunas):
    for nome_coluna in nome_colunas:
        scaler = StandardScaler()
        df[nome_coluna] = scaler.fit_transform(df[[nome_coluna]])
        joblib.dump(scaler, f"./objects/scaler{espaçamento+nome_coluna}.joblib")
    return df

In [51]:
X_test =save_scalers(X_test,
                     ['tempoprofissao',
                      'renda',
                      'idade',
                      'dependentes',
                      'valorsolicitado',
                      'valortotalbem',
                      'proporcaosolicitadototal'])

In [50]:
X_train =save_scalers(X_train,
                     ['tempoprofissao',
                      'renda',
                      'idade',
                      'dependentes',
                      'valorsolicitado',
                      'valortotalbem',
                      'proporcaosolicitadototal'])

In [52]:
X_test.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,proporcaosolicitadototal
119,Cientista de Dados,1.185897,1.825807,Própria,Ens.Fundamental,Baixo,0.188297,-0.166206,Casado,SpeedFury,0.919490,3.461002,-0.264215
128,Dentista,0.847874,-0.254147,Alugada,Superior,Justo,0.753189,0.831028,Víuvo,SpeedFury,4.597073,-0.767453,3.556824
135,Médico,-0.757734,-0.769935,Outros,Ens.Fundamental,MuitoBom,1.237382,-0.166206,Divorciado,TrailConqueror,-0.613239,0.412066,-0.525845
91,Programador,-0.588723,0.571378,Própria,PósouMais,Baixo,0.753189,0.831028,Víuvo,EcoPrestige,0.153154,1.174300,-0.278261
112,Programador,1.185897,-0.721346,Outros,Superior,Bom,-0.215197,-1.163439,Casado,DoubleDuty,-0.555359,-0.197721,-0.444277


Por que salvar objetos?
- Uso do modelo treinado em produção

### 5.3 Codificação de Categorias

- Algoritmos entendem números
- Categorical encoding é o processo de transformar categorias em números

Duas Formas:
- Label encoding
- One-hot encoding

5.3.1 Label encoding

Cada categoria recebe um número, normalmente em ordem alfabética.

In [53]:
# Codificação da variável classe
mapeamento = {'ruim': 0, 'bom': 1}
y_train = np.array([mapeamento[item] for item in y_train])
y_test  = np.array([mapeamento[item] for item in y_test])


In [55]:
espaçamento = '-'
def save_encoders(df, nome_colunas):
    for nome_coluna in nome_colunas:
        label_encoder = LabelEncoder()
        df[nome_coluna] = label_encoder.fit_transform(df[nome_coluna])
        joblib.dump(label_encoder, f"./objects/labelencoder{espaçamento+nome_coluna}.joblib")
    return df

In [56]:
X_train = save_encoders(X_train,['profissao',
                                 'tiporesidencia',
                                 'escolaridade',
                                 'score',
                                 'estadocivil',
                                 'produto'])

In [57]:
X_test= save_encoders(X_test,['profissao',
                                 'tiporesidencia',
                                 'escolaridade',
                                 'score',
                                 'estadocivil',
                                 'produto'])

In [58]:
X_train

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,proporcaosolicitadototal
79,2,1.628292,-1.007150,1,2,3,-0.237227,-0.951479,3,6,-0.004945,-0.173090,-0.176873
54,6,-0.312718,0.062261,0,0,3,1.608533,-0.951479,1,0,-0.568777,-0.706208,-0.226440
106,8,1.073718,-0.607772,2,1,3,1.182588,1.087404,3,0,-0.899311,-0.706208,-0.321025
90,6,0.796431,-0.417512,0,0,3,-0.024255,0.067963,2,3,-0.967338,-0.439649,-0.359797
145,7,1.258576,0.819025,2,3,1,1.182588,-0.951479,1,4,0.692582,2.225939,-0.271687
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,6,-1.606725,-1.150379,0,0,3,0.472680,-0.951479,1,4,0.175483,2.225939,-0.317928
89,8,0.149427,-1.279103,1,1,3,-0.876144,0.067963,2,1,1.672443,-1.860562,6.051030
65,4,-0.035431,-0.375291,1,3,3,-1.799024,-0.951479,3,7,-0.535125,-0.546273,-0.242472
80,1,0.334285,-0.409114,2,2,3,0.401689,0.067963,1,3,-0.353756,-0.439649,-0.213478


## 5.4 Seleção de Atributos

Um subconjunto de atributos pode criar um modelo com performance melhor
<span style="color: blue">Este texto está em azul</span>

In [59]:
# Seleção de Atributos
model = RandomForestClassifier()

# Instancia o RFE
selector = RFE(model, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)

# Transforma os dados
X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)
joblib.dump(selector, './objects/selector.joblib')


['./objects/selector.joblib']

In [60]:
print(selector.support_)
print(selector.ranking_)

[ True  True  True False False  True  True False  True  True  True  True
  True]
[1 1 1 3 2 1 1 4 1 1 1 1 1]
